#  Interest rate products

Technical notes can be skipped on a basic reading. These notes end when  $\diamondsuit$ appears.

## European Swaption

If the swap has tenor structure given by $\{T_\alpha,\ldots,T_\beta\}$, the payoff of a european payer swaption is given by, [Brigo and Mercurio, 2006] p.19 ss,
$$
    N\cdot \mathcal{A}(T_\alpha)(S_{\alpha,\beta}(T_\alpha)-K)^+\,,
$$
being $S_{\alpha,\beta}$ the swap rate and $\mathcal{A}$ defined below. For a receiver swaption,
$$
    N\cdot \mathcal{A}(T_\alpha)(K-S_{\alpha,\beta}(T_\alpha))^+\,.
$$
In general,
$$
\frac{f^\text{kind}(S_{\alpha,\beta}(T_\alpha), \mathcal{A}(T_\alpha))}{N\cdot \mathcal{A}(T_\alpha)}:=\left({S_{\alpha,\beta}(T_\alpha)}-K\right)\mathbb{1}_{\{S_{\alpha,\beta}(T_\alpha)\geq K\}} -\left({S_{\alpha,\beta}(T_\alpha)}-K\right)\delta_\text{kind,put}\,.
$$
For

* *Physical-settlement:* $\mathcal{A}(t):=A(t)$, the annuity.
* *Cash-settlement:* $\mathcal{A}(t):=a\left(S_{\alpha,\beta}(t)\right)$, see p.206 of Andersen and Piterbarg.

$\text{Code}$ - math $(\TeX)$ **translation**

* `maturity` $\rightarrow T_\alpha$,
* `swap_rate` $\rightarrow [S_{\alpha,\beta},\mathcal{A}]$, as a `SwapRate` (as in `MultiRate`),
* `strike` $\rightarrow K$,
* `kind` $\rightarrow$ $\text{kind}\in\{\text{call}, \text{put}\}$, for payer and receiver swaptions, respectively.
* `nominal` $\rightarrow N.$


**Remark.** As we are generating the prices from "rate curves" and the european swaption depends on the (simulated) annuity, so it is sensible to consider it as a rate and the swaption a product depending on two rates, in the same way that other products depend on several rates, given by the `MultiRate` class.

#### References

- Brigo, Damiano, and Fabio Mercurio. Interest Rate Models - Theory and Practice. Springer, 2006.

#### Technical note (`payoff` method) 

We analyze in detail the input and output of the `payoff` method:

$\bullet$ For the observation dates $\{t^1_j\}_{j=0}^{J}$ and the valuation dates $\{t^0_k\}_{k=0}^{K}$, the *swap rate* associated with the $i$-th simulation of the $l$-th component is a numpy.ndarray denoted by
    
$$\textrm{price}[i,j,k,l]=S_{i, t_k^0}^l(t^1_j)\,,$$
where, for the sake of simplicity, we have removed the subscripts $\alpha,\beta.$ Recall that $l=0$ corresponds to $S_{\alpha,\beta}$ and $l=1$ with the "rate" $\mathcal{A}$.
    
$\bullet$  `dates_dic` is a dictionary (or pandas.Series) which assigns an index to each date. The observation dates $\{t^1_j\}_{j=0}^{J}$ satisfy $\{t^1_j\}_{j=0}^{J}\subseteq \{\tau_{j'}\}_{j'=0}^{J'}$, where $\{\tau_{j'}\}_{j'=0}^{J'}$ are the simulation dates. That is to say, we generally simulate the prices for more dates than the ones we need for a particular product. In this sense, `dates_dic` can be understood as the map: $\tau_{j'} \mapsto j'$. In particular, $t^1_j \mapsto j'$ such that $t^1_j = \tau_{j'}.$

$\bullet$ `n` is an integer indicating the number of observation dates previous to a fixed observation date. This is needed in the Monte Carlo engine for exotic (path-dependent) products. For a given  `n` we get the value of the underlying for the previous `n` observation dates and simulate it for the new observation dates. For more details see the method `generate_paths_for_pricing` in *mc_engines.py*.

The payoff function returns a numpy.ndarray of **3 dimensions** and shape $(|\mathcal{I}|,|\mathcal{J}|-n,|\mathcal{K}|)$, where $|\mathcal{I}|:= I+1$, $|\mathcal{J}|:= J+1$ and $|\mathcal{K}|:=K+1$ are the numbers of elements in the indexes $i$, $j$ and $k$. Note that we give the payoff excluding the previous observation dates. The output satisfies

$$\textrm{payoff}[i,j,k]=f\left(S_{i, t_k^0}^{0}(t^1_j), S_{i, t_k^0}^{1}(t^1_j)\right)\times\delta_{j, j_T-n}\,,$$

where $\delta_{i,j}$ denotes the Kronecker delta and $j_T$ satisfies $t^1_{j_T} = T$. 
 
Finally, note that for every product we are assuming a long position, for a short position we would have $P_T = -f(S)$. $\diamondsuit$

$\text{Code}$ - math $(\TeX)$ **translation**

* `price`      $\rightarrow$ $[S_{\alpha,\beta}, \mathcal{A}]$ as described above (same structure as in the `MultiRate` class),
* `dates_dic`     $\rightarrow$ Map assigning an index $j$ to each date, 
* `n`  $\rightarrow$ Number of observation dates previous to a given observation date (needed for the Monte Carlo engine, **it is used?**).

## GeneralSwap

### Description

Given the tenor structure for each leg, $\mathcal{T}^f=\{T_\alpha,\ldots, T^f_l,\ldots,T_\beta\}$ for $f\in\{\text{float, fixed}\}$ with

* *payment dates*: $\{t_j^{p,f}\}_{j=0}^{J^f}\subset \mathcal{T^f},$ payment dates for each leg. The union gives $\{t_j^{p}\}_{j=0}^{J^p}\subset \mathcal{T},$
* *fixing dates* (observation dates): $\{t_j^1\}_{j=0}^{J}\subset \mathcal{T}^\text{float}\,,$

where $\mathcal{T}:=\bigcup_f\mathcal{T}^f=\mathcal{T}^\text{float}\cup\mathcal{T}^\text{fixed}$, and the underlying rate $R$, the payoff (long position) is given by

$$
P_j(t_j^p) = f_j\left(\tilde{R}(t_j^p)\right)=N_j\cdot\left(\delta_{t^p_j, \mathcal{T}^\text{float}}\left(\tau^\text{float}_j\cdot\tilde{R}(t_j^p)\right)  - \delta_{t^p_j, \mathcal{T}^\text{fixed}}\left(\tau^\text{fixed}_j\cdot K_j\right)\right)\,,
$$
being $N_j$ the, possibly varying, nominal. Obviously,
$$
\delta_{t^p_j, \mathcal{T}^f} = \sum_{l=0}^{L^f} \delta_{t^p_j, {T}^f_l}\,.
$$
This function allows us to extend trivially arrays that are defined only for one leg to the union of both legs, as $K$. With some *abuse of notation*, we will use the same names for both vectors.
Also, we assume there is a bijection between $\{t^1_j\}_{j=0}^J$ and $\{t^{p,\text{float}}_j\}_{j=0}^J$ so
$$
\tilde{R}(t^{p,\text{float}}_j):=R(t^{1}_j)\,.
$$
Several remarks are in order:

* The strike and nominal are not necessarily constant, so *amortizing* and *accreting* swaps are included.
* $R$ can be the difference of two rates with $K_j=0$ using the `DifferenceRate` class, so *exotic* *swaps* are included. E.g., the difference between a CMS and Libor. See Section 5.13 of Andersen and Piterbarg.
* Payment dates and observation (fixing) dates are not necessarily the same, so the "*in-arrears*" and "*fixed-in-advance*" (standard) cases are included.
* The function $\delta_{t^1_j, \mathcal{T}^f}$ is implemented in the code under the name of `delta_indexing`.

$\text{Code}$ - math $(\TeX)$ **translation**
* `underlying` $\rightarrow R$,
* `obsdates` $\rightarrow \{t^1_j\}_{j=0}^J$ ,
* `effective_date` $\rightarrow$ $T_\alpha^{\text{float}}=T_\alpha^{\text{fixed}}$ (it is assumed that both are the same),
* `end_date` $\rightarrow$ $T_\beta^{\text{float}}=T_\beta^{\text{fixed}}$ (it is assumed it is the same),
* `floating_freq` $\rightarrow 12\cdot\left(T_l^{\text{float}}-T_{l-1}^{\text{float}}\right)$ for $l\in\{\alpha^{\text{float}}+1, \ldots, \beta^{\text{float}}\}$ (it is assumed that this is constant). For instance, if floating_freq=6, then, approximately (depends on the day count convention), $\left(T_l^{\text{float}}-T_{l-1}^{\text{float}}\right)=0.5$,
* `fixed_freq` $\rightarrow 12\cdot\left(T_l^{\text{fixed}}-T_{l-1}^{\text{fixed}}\right)$, as above,
* `legs_calendar` $\rightarrow [\text{DCC}^{\text{float}}, \text{DCC}^{\text{fixed}}]$ or $\text{DCC}$. In the latter case, the same calendar for each leg.
* `strike` $\rightarrow [K_j]_{j=0}^{J^\text{fixed}}\,$ (after possibly extending it to all the necessary dates). By _default_, it equals 0,
* `nominal` $\rightarrow [N_j]_{j=0}^{J^p}\,$ (after possibly extending it to all the necessary dates).

The class `SwapRate` is used to define some attributes. Also, we assume that:

* `pay_dates` $\rightarrow \{t^p_j\}_{j=0}^{J^p}=\mathcal{T}\backslash \{T_\alpha\}$, which cover the the "*in-arrears*" and "*fixed-in-advance*" (standard) cases. Note that the first date is needed for $\tau$.


#### Technical note (`payoff` method)

For the conventions and details on the price and payoff arrays see the "Technical note (`payoff` method)" in the [European Swaption section](##European-Swaption)

The payoff function in this case is

$$\textrm{payoff}[i,j,k]=f_j(R_{i, t_k^0}(t^p_j))\,,$$

where $f_j(R)$ was defined above. $\diamondsuit$

Explicitly,

$\bullet$ For the simulation dates $\{t^s_{j}\}_{j=0}^{J'}$ and the valuation dates $\{t^0_k\}_{k=0}^{K}$, the *rate* associated with the $i$-th simulation of the $l$-th component is a numpy.ndarray denoted by
    
$$\textrm{price}[i,j,k,l]=R_{i, t_k^0}^l(t^s_j)\,,$$
    
$\bullet$  `dates_dic` is a dictionary (or pandas.Series) which assigns an index to each date. The observation dates $\{t^1_j\}_{j=0}^{J}$ satisfy $\{t^1_j\}_{j=0}^{J}\subseteq \{t^s_{j'}\}_{j'=0}^{J'}$, where $\{t^s_{j'}\}_{j'=0}^{J'}$ are the simulation dates. That is to say, we generally simulate the prices for more dates than the ones we need for a particular product. In this sense, `dates_dic` can be understood as the map: $t^s_{j'} \mapsto j'$. In particular, $t^1_j \mapsto j'$ such that $t^1_j = t^s_{j'}.$

$\bullet$ `n` is an integer indicating the number of observation dates previous to a fixed observation date. This is needed in the Monte Carlo engine for exotic (path-dependent) products. For a given  `n` we get the value of the underlying for the previous `n` observation dates and simulate it for the new observation dates. For more details see the method `generate_paths_for_pricing` in *mc_engines.py*.

The payoff function returns a numpy.ndarray of **3 dimensions** and shape $(|\mathcal{I}|,|\mathcal{J}|-n,|\mathcal{K}|)$, where $|\mathcal{I}|:= I+1$, $|\mathcal{J}|:= J+1$ and $|\mathcal{K}|:=K+1$ are the numbers of elements in the indexes $i$, $j$ and $k$. Note that we give the payoff excluding the previous observation dates. The output satisfies

$$\textrm{payoff}[i,j,k]=f_j(R_{i, t_k^0}(t^p_j))\,,$$

where $f_j(R)$ was defined above. 
 
Finally, note that for every product we are assuming a long position, for a short position we would have $P_T = -f(S)$. $\diamondsuit$

$\text{Code}$ - math $(\TeX)$ **translation**
* `price`      $\rightarrow$ $R$ as described above ,
* `dates_dic`     $\rightarrow$ Map assigning an index $j$ to each date, 
* `n`  $\rightarrow$ Number of observation dates previous to a given observation date (needed for the Monte Carlo engine, **it is used?**).

### (Vanilla) Libor Swap

In this case, $t^1_j=t^{p, \text{ float}}_{j-1}$ and the rate equal to the Libor rate.

### Libor-in-arrears Swap

In this case, $t^1_j=t^{p, \text{ float}}_j$ and the rate equal to the Libor rate.

###  Constant Maturity Swap (CMS Swap)

In this special case $R$ is a swap rate is of the form $S_{j,j+c}$, where $c$ is the constant maturity.

### Amortizing swaps

$N_j$ is no longer constant.

### TARN(GeneralSwap)

$R$ is an element of the ``TARNRate``.

## CapFloor

Given a tenor structure $\{T_\alpha,\ldots,T_\beta\}$ with $\{t_j^p\}_{j=0}^J$, $\{t_j^1\}_{j=0}^J\subset \{T_\alpha,\ldots,T_\beta\}$ the payment and fixing dates, respectively, and the underlying rate $R$, for a long position, the payoff is given by

$$
P_j(t_j^p) = f_j\left(R(t_j^1)\right)=N\cdot\tau_j\cdot\left(\left( R(t_j^1)  - K_j\right)_+ - \left( R(t_j^1)  - K_j\right)\cdot\delta_{\text{kind, floor}}\right).
$$

being $N$ the nominal, $\mathcal{T}^f$ the tenor structure of the $f$-leg of the swap, either float or fixed.

$\text{Code}$ - math $(\TeX)$ **translation**

* `underlying` $\rightarrow R$,
* `obsdates` $\rightarrow \{t^1_j\}_{j=0}^J$ ,
* `calendar` $\rightarrow$ calendar for computations, $\tau_j=\text{DCC}(t_{j-1}^p,t_j^p)$,
* `strike` $\rightarrow [K_j]_{j=0}^J\,,$
* `nominal` $\rightarrow [N_j]_{j=0}^J\,,$
* `pay_dates` $\rightarrow \{t^p_j\}_{j=0}^J$, by _default_ $t^1_j=t^p_j$,
* `tenor_dates ` $\rightarrow \{T_j\}_{j=0}^{J+1},$ if it is not specified, it is $\{t^1_j\}_{j=0}^J\cup\{t^p_j\}_{j=0}^J$. Either `pay_dates` or `tenor_dates` must be specified.

#### Technical note (`payoff` method)

For the conventions and details on the price and payoff arrays see the "Technical note (`payoff` method)" in the [previous section](#european-swaption).
The payoff function in this case is

$$\textrm{payoff}[i,j,k]=f_j(R_{i, t_k^0}(t^1_j))$$

where $f_j(R)$ was defined above. $\diamondsuit$

## RangeAccrual

Given two rates $R, X$, a set of payment dates $\{t^p_n\}_{n=0}^N$, a set of reference dates $\{t^1_{j}\}_{j=0}^J$, and an interval $[l, u]$, a long position of the payoff is given by

$$
P_n(t_{n+1}^p) = f_n\left(R(t_n^p), \left(X(t^1_{j})\right)_{j\in\mathcal{J}_n}\right)=N\cdot R(t_n^p)\times \dfrac{\#\{j\in\mathcal{J}_n~\mid~X(t_j^1)\in[l,u]\}}{\# \mathcal{J}_n}.
$$

being $N$ the nominal and $\mathcal{J}_n:=\left\{j\in\{0,\ldots, J\}\,\mid\, t^1_j\in[t^p_n, t^p_{n+1}\mid \right\}$, being $\mid$ either $``]"$ or $``)"$. 

$\text{Code}$ - math $(\TeX)$ **translation**

* `underlying` $\rightarrow [R, X]$ as a `MultiRate`,
* `calendar` $\rightarrow$ calendar for computations, $\tau_j=\text{DCC}(t_{j-1}^p,t_j^p)$,
* `nominal` $\rightarrow N\,,$
* `border` $\rightarrow$ `True` if the right end of the interval is included, $\mid$ equals $``]"$, and `False` otherwise. By _default_, it is `False`.
With this, we can define:
* `reference_dates_dic` $\rightarrow \{t^p_n : \{t^1_{j}\}_{j\in\mathcal{J}_n} \}_{n=0}^N$ .

#### Technical note (`payoff` method)

For the conventions and details on the price and payoff arrays see the "Technical note (`payoff` method)" in the [previous section](#european-swaption).

The payoff function in this case is

$$\textrm{payoff}[i,j,k]=f_j\left(R_{i, t_k^0}(t^1_j), \left(X_{i, t_k^0}(t^1_{j'})\right)_{j'\in\mathcal{J}_n}\right)\,,$$

where $f_j(R)$ was defined above. $\diamondsuit$

## CMSSpreadForward

Given two rates $R_0, R_1$, a set of payment dates $\{t^p_n\}_{n=0}^N$, a set of reference dates $\{t^1_{j}\}_{j=0}^J$, 

$$
P_n(t_{n+1}^p) = f^\text{Forward}\left(R_1(t_n^p)-R_0(t_n^p)\right)
$$

$\text{Code}$ - math $(\TeX)$ **translation**

* `underlying` $\rightarrow [R, X]$ as a `MultiRate`,
* `calendar` $\rightarrow$ calendar for computations, $\tau_j=\text{DCC}(t_{j-1}^p,t_j^p)$,
* `nominal` $\rightarrow N\,,$
* `border` $\rightarrow$ `True` if the right end of the interval is included, $\mid$ equals $]$, and `False` otherwise. By _default_, it is `False`.

#### Technical note (`payoff` method)

For the conventions and details on the price and payoff arrays see the "Technical note (`payoff` method)" in the [previous section](#european-swaption).

The payoff function in this case is

$$\textrm{payoff}[i,j,k]=f_j(R_{i, t_k^0}(t^1_j))$$

where $f_j(R)$ was defined above. $\diamondsuit$